# Recommendation System

In this recommendation system I have used Kaggle dataset named Post Recommendation System Here is the link of that : https://www.kaggle.com/vatsalparsaniya/post-pecommendation

It will consume a user_id and check the history or like or vote he has given to post and check other users simliarities and then recommend posts.

Here we go::::::::::::

In [ ]:
#importing relevant libraries
%pip install pandas
%pip install numpy
import pandas as pd
import numpy as np
import re

In [ ]:
#Datasets Importing 
#post_data consists of post details like post_id, title, category
#user_data consists of user's information
#view_data consists of which user view which post

post_data = pd.read_csv(r'postnew_data (1).csv')
user_data = pd.read_csv(r'user_data (2).csv')
view_data = pd.read_csv(r'view_data (1).csv')

# New section

#drop the irrevelent features i.e category and title is important but not for this.

In [ ]:
post_data = post_data.drop(['title','category'], axis=1)

In [ ]:
post_data.head()

In [ ]:
user_data.head()

In [ ]:
view_data.head()

In [ ]:
post_data.shape

In [ ]:
post_data.head()

In [ ]:
#Making Recommendation system is not a supervised learning right that consist of label but it is 
# unsupervised learning so we have to add likes or make it valuable that means the user rate it valuable
# in terms of rating like out of 5 okay
# I add up it.

dataframe = pd.DataFrame(view_data)

In [ ]:
dataframe["Valuable"] = np.random.randint(1, 6, len(dataframe))

In [ ]:
#this is our data right now.
dataframe

In [ ]:
# in this cell we are merging view data and the dataframe I have made of post data. in this post id is common.

df = pd.merge(dataframe, post_data ,on='post_id')
df.tail()

In [ ]:
df.head()

In [ ]:
#drop the irrevelent features time_stamp is not 
#important right now.
data = df.drop('time_stamp', axis=1)

In [ ]:
data.tail()

In [ ]:
data.shape

In [ ]:
# In this first we have to make a column here that consists of total ratings a content or a post have. 
#like 10 Funny ART Quotes has 15 votes given by users.

combine_post_rating = data.dropna(axis = 0, subset = ['content'])

post_ratingCount = (combine_post_rating.
     groupby(by = ['content'])['Valuable'].
     count().
     reset_index().
     rename(columns = {'Valuable': 'totalValuableCount'})
     [['content', 'totalValuableCount']]
    )
post_ratingCount.head() 

In [ ]:

#Merge it into our main dataset 
rating_with_totalValuableCount = combine_post_rating.merge(post_ratingCount, left_on = 'content', right_on = 'content', how = 'left')
rating_with_totalValuableCount.tail()

In [ ]:
#It is just how to see data desciption and stuff.
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(post_ratingCount['totalValuableCount'].describe())

In [ ]:
print(post_ratingCount['totalValuableCount'].quantile(np.arange(.9, 1, .01)))

In [ ]:
#we have to fix a threshold here that machine will recommend post to user that have votes greater than
# the threshold values
popularity_threshold = 13
rating_popular_post = rating_with_totalValuableCount.query('totalValuableCount >= @popularity_threshold')
rating_popular_post.tail()

In [ ]:
rating_popular_post.shape

In [ ]:
user_data.head()

In [ ]:
len(user_data.city.unique())

Making model from here.
we have to make a data matrix like indexes are titles and columns are user id and target is how valuable or how much likes it have.

Use here cosine similiarity to find the simliarity between user and other users.

In [ ]:
%pip install scikit-learn
%pip install scipy
from scipy.sparse import csr_matrix
rating_popular_post = rating_popular_post.drop_duplicates(['user_id', 'content'])
rating_popular_post_pivot = rating_popular_post.pivot(index = 'content', columns = 'user_id', values = 'Valuable').fillna(0)
rating_popular_post_matrix = csr_matrix(rating_popular_post_pivot.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(rating_popular_post_matrix)

In [ ]:
rating_popular_post_pivot.shape[0]

In [ ]:
type(rating_popular_post_pivot)

In [ ]:
view_data

Choose a random users and recommend it a content of post by looking their cosine simliarity scores

In [ ]:
rating_popular_post_pivot
rating_popular_post_pivot.to_csv("rating_popular_post_pivot.csv")
rating_popular_post_pivot

In [ ]:
df = pd.read_csv("rating_popular_post_pivot.csv")
df

In [ ]:
query_index = np.random.choice(rating_popular_post_pivot.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(rating_popular_post_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

In [ ]:
# this is our name of user 3 and posts he has seen or vote it and our task is to recommend it similiar
#post
rating_popular_post_pivot.index[query_index]

In [ ]:
#Here we go
for i in range(1, len(distances.flatten())):
    print(rating_popular_post_pivot.index[indices.flatten()[i]])

In [ ]:

%pip install flask
%pip install flask_cors
import numpy as np 
from flask import Flask, request, jsonify 
from flask_cors import CORS, cross_origin 
import pickle 
import json 

app = Flask(__name__)   
CORS(app, support_credentials=True) 

@app.route('/predict',methods=['POST']) 
@cross_origin(supports_credentials=True) 
def predict(): 
    data = request.get_json(force=True) 
    print(data) 
#     uid = "5eece14ffc13ae660900008b"
#     df = view_data[(view_data["user_id"]==uid)]
    # df = df.sort_values("Valuable", ascending=False)
#     pid = df["post_id"][0]
#     pid            
    r = {"result" : 1 } 
    response = jsonify(r) 
    return response          

if __name__ == '_main_': 
    app.run(port=5000, debug=True)
# for i in range(1, len(distances.flatten())):
#       print('{0}: {1}, with distance of {2}:'.format(i, rating_popular_post_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

In [ ]:

# # import model


# import re

# # Define a regular expression pattern to match non-printable characters
# non_printable = re.compile('[\x00-\x08\x0B\x0C\x0E-\x1F\x7F-\x9F]')

# # Open the file for reading and writing
# # with open('checknonprint.txt', 'r') as input_file, open('output_file.txt', 'w') as output_file:
#     # Loop over each line in the input file
#     # for line in input_file:
#     #     # Remove any non-printable characters from the line
#     #     clean_line = non_printable.sub(' ', line)
#     #     # Write the cleaned line to the output file
#     #     output_file.write(clean_line)

# import pickle

# # Load the trained model object
# model_object = model.model_knn()

# # Serialize the object and write it to a pickled file
# with open('model.pickle', 'wb') as f:
#   for line in f:
#         # Remove any non-printable characters from the line
#         clean_line = non_printable.sub(' ', line)
#         # Write the cleaned line to the output file
#         # output_file.write(clean_line)
#         pickle.dump(model_object,clean_line)




import pickle

with open('model_knn.pkl', 'wb') as f:
    pickle.dump(model_knn, f)


 